### Object Storage secrets import

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('/Users/p.miroshin/Documents/Masters/2d_year/keys.env')

ACCESS_SECRET_KEY = os.getenv("ACCESS_SECRET_KEY")
ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")

### Object Storage test functions

In [4]:
from object_storage import (  # Ваши функции находятся в модуле object_storage
    read_table_from_s3_csv,
    write_table_to_s3_csv,
    get_old_timestamp,
    get_current_timestamp,
    check_user_exists,
    add_user,
    update_last_updated,
    add_shiki_to_user,
    load_user_anime_list,
    get_context_data,
    generate_id
)
import pandas as pd

In [2]:
import json

In [8]:
recommendations = [
    {"название": "Noumin Kanren no Skill bakka Agetetara Nazeka Tsuyoku Natta."},
    {"название": "Isekai de Cheat Skill wo Te ni Shita Ore wa, Genjitsu Sekai wo mo Musou Suru: Level Up wa Jinsei wo Kaeta"},
    {"название": "Sousou no Frieren"}
]


In [1]:
from yandex_gpt_alpha import *

In [8]:
user_data = {
    'top_genre': 'Action',
    'median_episodes': 12.0,
    'top_titles': ['Grand Blue', 'Fullmetal Alchemist', 'Fullmetal Alchemist: Brotherhood'],
    'bottom_titles': [
        'Noumin Kanren no Skill bakka Agetetara Nazeka Tsuyoku Natta.',
        'Isekai de Cheat Skill wo Te ni Shita Ore wa, Genjitsu Sekai wo mo Musou Suru: Level Up wa Jinsei wo Kaeta',
        'Tokyo Ghoul √A'
    ]
}

# Запрос пользователя
user_query = "Хочу посмотреть аниме с  интересной драмой"

# Вызов функции
result = get_anime_recommendations(user_data, user_query)

# Вывод результата
print(result)

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': '```\n[\n    {"1": "Attack on Titan"},\n    {"2": "Berserk"},\n    {"3": "Vinland Saga"}\n]\n```'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '294', 'completionTokens': '37', 'totalTokens': '331'}, 'modelVersion': '23.10.2024'}}


In [2]:
# Пример исходных данных от API
api_response = {
    'result': {
        'alternatives': [
            {
                'message': {
                    'role': 'assistant',
                    'text': '```\n[\n    {"1": "Boku no Hero Academia"},\n    {"2": "My Hero Academia: Season 5"},\n    {"3": "Attack on Titan"}\n]\n```'
                },
                'status': 'ALTERNATIVE_STATUS_FINAL'
            }
        ],
        'usage': {
            'inputTextTokens': '298',
            'completionTokens': '43',
            'totalTokens': '341'
        },
        'modelVersion': '23.10.2024'
    }
}

# Вызов функции для обработки ответа
recommendations = format_recommendations(api_response)

# Вывод результата
print(recommendations)

[{'название': 'Boku no Hero Academia'}, {'название': 'My Hero Academia: Season 5'}, {'название': 'Attack on Titan'}]


In [11]:
from main import *

In [15]:
chat_id = 505096036
title = 'test'

In [29]:
shikimori_username

np.int64(1018441)

In [4]:
def success_response(data=None, message="Успешный запрос.", code=200):
    """
    Формирует успешный JSON-ответ.
    """
    return {
        "status": "success",
        "message": message,
        "data": data,
        "code": code
    }

def error_response(message="Произошла ошибка.", code=400):
    """
    Формирует JSON-ответ при ошибке.
    """
    return {
        "status": "error",
        "message": message,
        "data": None,
        "code": code
    }



ADMINS = [505096036, 123456789]


def is_admin_user(chat_id):
    """
    Проверяет, является ли пользователь администратором.
    """
    return chat_id in ADMINS



def create_recommended_list(chat_id, name, description, list_type, genre, year, anime_ids, created_by):
    """
    Создание нового рекомендованного списка.
    """

    # Проверяем права администратора
    if not is_admin_user(chat_id):
        return error_response("У вас нет прав для выполнения этой операции.", code=401)

    # Загружаем данные
    recommended_lists = read_table_from_s3_csv('recommended_lists.csv')
    recommended_list_items = read_table_from_s3_csv('recommended_list_items.csv')

    # Проверяем дублирование имени
    if not recommended_lists.empty and (recommended_lists['name'] == name).any():
        return error_response(f"Список с названием '{name}' уже существует.", code=409)

    # Создаем новый список
    list_id = generate_id()
    new_list = pd.DataFrame([{
        'list_id': list_id,
        'name': name,
        'description': description,
        'type': list_type,
        'genre': genre,
        'year': year,
        'created_by': created_by,
        'created_at': get_current_timestamp().isoformat(),
        'updated_at': get_current_timestamp().isoformat()
    }])
    recommended_lists = pd.concat([recommended_lists, new_list], ignore_index=True)

    # Добавляем элементы списка
    new_items = pd.DataFrame([{
        'list_id': list_id,
        'anime_id': anime_id
    } for anime_id in anime_ids])
    recommended_list_items = pd.concat([recommended_list_items, new_items], ignore_index=True)

    # Сохраняем изменения
    write_table_to_s3_csv(recommended_lists, 'recommended_lists.csv')
    write_table_to_s3_csv(recommended_list_items, 'recommended_list_items.csv')

    return success_response(
        message=f"Список '{name}' успешно создан.",
        data={"list_id": list_id},
        code=201
    )



def get_recommended_list_by_name(name):
    recommended_lists = read_table_from_s3_csv('recommended_lists.csv')
    recommended_list_items = read_table_from_s3_csv('recommended_list_items.csv')

    recommended_list = recommended_lists[recommended_lists['name'] == name]

    if recommended_list.empty:
        return error_response(f"Список с названием '{name}' не найден.", code=404)

    # Извлекаем ID списка и связанные тайтлы
    list_id = recommended_list.iloc[0]['list_id']
    list_items = recommended_list_items[recommended_list_items['list_id'] == list_id]
    anime_ids = list_items['anime_id'].tolist()

    result = {
        "list_id": list_id,
        "name": name,
        "description": recommended_list.iloc[0]['description'],
        "type": recommended_list.iloc[0]['type'],
        "genre": recommended_list.iloc[0]['genre'],
        "year": recommended_list.iloc[0]['year'],
        "anime_ids": anime_ids
    }

    return success_response(data=result)



def update_recommended_list(chat_id, list_id, name=None, description=None, anime_ids=None):
    """
    Обновление рекомендованного списка.
    """

    # Проверяем права администратора
    if not is_admin_user(chat_id):
        return error_response("У вас нет прав для выполнения этой операции.", code=401)

    # Загружаем данные
    recommended_lists = read_table_from_s3_csv('recommended_lists.csv')
    recommended_list_items = read_table_from_s3_csv('recommended_list_items.csv')

    # Проверяем существование списка
    if list_id not in recommended_lists['list_id'].values:
        return error_response(f"Список с ID '{list_id}' не найден.", code=404)

    # Обновляем данные
    if name:
        recommended_lists.loc[recommended_lists['list_id'] == list_id, 'name'] = name
    if description:
        recommended_lists.loc[recommended_lists['list_id'] == list_id, 'description'] = description

    recommended_lists.loc[recommended_lists['list_id'] == list_id, 'updated_at'] = get_current_timestamp().isoformat()

    # Обновляем элементы списка
    if anime_ids is not None:
        # Удаляем старые элементы и добавляем новые
        recommended_list_items = recommended_list_items[recommended_list_items['list_id'] != list_id]
        new_items = pd.DataFrame([{
            'list_id': list_id,
            'anime_id': anime_id
        } for anime_id in anime_ids])
        recommended_list_items = pd.concat([recommended_list_items, new_items], ignore_index=True)

    # Сохраняем изменения
    write_table_to_s3_csv(recommended_lists, 'recommended_lists.csv')
    write_table_to_s3_csv(recommended_list_items, 'recommended_list_items.csv')

    return success_response(message=f"Список с ID '{list_id}' успешно обновлен.")


def compare_list_with_user(list_id, chat_id):
    user_anime = read_table_from_s3_csv('user_anime.csv')
    recommended_list_items = read_table_from_s3_csv('recommended_list_items.csv')

    # Проверяем существование списка
    list_items = recommended_list_items[recommended_list_items['list_id'] == list_id]
    if list_items.empty:
        return error_response(f"Список с ID '{list_id}' не найден.", code=404)

    list_anime_ids = set(list_items['anime_id'].tolist())

    # Проверяем просмотренные аниме пользователя
    user_anime_list = user_anime[user_anime['chat_id'] == chat_id]
    if user_anime_list.empty:
        return error_response(f"У пользователя с ID '{chat_id}' нет просмотренных аниме.", code=404)

    user_anime_ids = set(user_anime_list['anime_id'].tolist())

    viewed = list(list_anime_ids & user_anime_ids)
    not_viewed = list(list_anime_ids - user_anime_ids)

    return success_response(data={
        "viewed": viewed,
        "not_viewed": not_viewed
    })




def get_recommended_list_by_name(name):
    recommended_lists = read_table_from_s3_csv('recommended_lists.csv')
    recommended_list_items = read_table_from_s3_csv('recommended_list_items.csv')

    recommended_list = recommended_lists[recommended_lists['name'] == name]

    if recommended_list.empty:
        return error_response(f"Список с названием '{name}' не найден.", code=404)

    # Извлекаем ID списка и связанные тайтлы
    list_id = recommended_list.iloc[0]['list_id']
    list_items = recommended_list_items[recommended_list_items['list_id'] == list_id]
    anime_ids = list_items['anime_id'].tolist()

    result = {
        "list_id": list_id,
        "name": name,
        "description": recommended_list.iloc[0]['description'],
        "type": recommended_list.iloc[0]['type'],
        "genre": recommended_list.iloc[0]['genre'],
        "year": recommended_list.iloc[0]['year'],
        "anime_ids": anime_ids
    }

    return success_response(data=result)



def get_all_recommended_lists():
    recommended_lists = read_table_from_s3_csv('recommended_lists.csv')

    if recommended_lists.empty:
        return success_response(data=[], message="Списков нет.", code=200)

    result = recommended_lists.to_dict(orient='records')
    return success_response(data=result)



def delete_recommended_list(chat_id, list_id):
    """
    Удаление рекомендованного списка.
    """

    # Проверяем права администратора
    if not is_admin_user(chat_id):
        return error_response("У вас нет прав для выполнения этой операции.", code=401)

    # Загружаем данные
    recommended_lists = read_table_from_s3_csv('recommended_lists.csv')
    recommended_list_items = read_table_from_s3_csv('recommended_list_items.csv')

    # Проверяем существование списка
    if list_id not in recommended_lists['list_id'].values:
        return error_response(f"Список с ID '{list_id}' не найден.", code=404)

    # Удаляем список и связанные элементы
    recommended_lists = recommended_lists[recommended_lists['list_id'] != list_id]
    recommended_list_items = recommended_list_items[recommended_list_items['list_id'] != list_id]

    # Сохраняем изменения
    write_table_to_s3_csv(recommended_lists, 'recommended_lists.csv')
    write_table_to_s3_csv(recommended_list_items, 'recommended_list_items.csv')

    return success_response(message=f"Список с ID '{list_id}' успешно удалён.")

In [ ]:
# Создание списка
response = create_recommended_list(
    name="Лучшие аниме для новичков",
    description="Подборка аниме для тех, кто только начал их смотреть.",
    list_type="Для новичков",
    genre=None,
    year=None,
    anime_ids=[101, 102, 103],
    created_by="admin"
)
print(response)

# Получение списка
response = get_recommended_list_by_name("Лучшие аниме для новичков")
print(response)

# Обновление списка
response = update_recommended_list(
    list_id="some_list_id",
    name="Новое название",
    description="Новое описание",
    anime_ids=[103, 104, 105]
)
print(response)

# Сравнение с просмотренными
response = compare_list_with_user(list_id="some_list_id", chat_id=505096036)
print(response)

# Вывод всех списков
response = get_all_recommended_lists()
print(response)

In [ ]:
response = create_recommended_list(
    chat_id=505096036,  # Пользователь с админскими правами из `ADMINS`
    name="Test",
    description="Список лучших боевых аниме.",
    list_type="Action",
    genre="Action",
    year=2023,
    anime_ids=[101, 102, 103],
    created_by=505096036
)
print(response)

In [42]:
update_recommended_list(
    chat_id=505096036,
    list_id=1,
    name="Новое название",
    description="Новое описание",
    anime_ids=[103, 104, 105]
)

{'status': 'success',
 'message': "Список с ID '1' успешно обновлен.",
 'data': None,
 'code': 200}

In [5]:
from object_storage import *

In [20]:
s3 = get_s3_client()

In [ ]:
response = s3.get_object(Bucket='bucket-first', Key='recommended_lists.csv')

In [21]:
pd.read_csv(response['Body'])

,list_id,name,description,type,genre,year,created_by,created_at,updated_at
0,1,Легендарные аниме,Культовые аниме всех времён,Общий,Action,NaN,505096036,2024-12-22T12:51:49.268934,2024-12-22T12:51:49.268938
1,2,Аниме для новичков,Идеальные тайтлы для начала знакомства с аниме,Для новичков,NaN,NaN,505096036,2024-12-22T12:51:49.268939,2024-12-22T12:51:49.268939
2,3,Лучшие комедии,Весёлые комедийные аниме,Комедия,Comedy,2023.0,123456789,2024-12-22T12:51:49.268940,2024-12-22T12:51:49.268941


In [3]:
import json

In [40]:
from delete_recommended_list import delete_list_handler

In [49]:
event_delete_list = {
    "httpMethod": "DELETE",
    "pathParameters": {
        "list_id": "2a6-24b58bc1e635"  # ID списка, который мы удаляем
    },
    "headers": {
        "Content-Type": "application/json"
    },
    "body": json.dumps({
        "auth_key": 123  # Ключ администратора для авторизации
    })
}
context_delete_list = {}

In [50]:
# Тестируем создание списка
response = delete_list_handler(event_delete_list, context_delete_list)

In [51]:
response

{'statusCode': 401,
 'body': '{"status": "error", "message": "У вас нет прав для выполнения этой операции.", "data": null}'}

In [45]:
get_all_recommended_lists()

{'status': 'success',
 'message': 'Успешный запрос.',
 'data': [{'list_id': '1',
   'name': 'Новое название',
   'description': 'Новое описание',
   'type': 'Общий',
   'genre': 'Action',
   'year': nan,
   'created_by': 505096036,
   'created_at': '2024-12-22T12:51:49.268934',
   'updated_at': '2024-12-22T13:06:43.348735+00:00'},
  {'list_id': '2',
   'name': 'Аниме для новичков',
   'description': 'Идеальные тайтлы для начала знакомства с аниме',
   'type': 'Для новичков',
   'genre': nan,
   'year': nan,
   'created_by': 505096036,
   'created_at': '2024-12-22T12:51:49.268939',
   'updated_at': '2024-12-22T12:51:49.268939'},
  {'list_id': '3',
   'name': 'Лучшие комедии',
   'description': 'Весёлые комедийные аниме',
   'type': 'Комедия',
   'genre': 'Comedy',
   'year': 2023.0,
   'created_by': 123456789,
   'created_at': '2024-12-22T12:51:49.268940',
   'updated_at': '2024-12-22T12:51:49.268941'},
  {'list_id': '6c9aafe2-6e6c-42cc-b148-09de54da817a',
   'name': 'Лучшие приключения